### File Rename

This notebook is used to clean renaming files.

In [12]:
import unicodedata
import os
#import os.path
import sys
import re
import numpy as np
import pandas as pd
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join

In [13]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [14]:
folder_name = "Turkish"  # English, Turkish, German, French, Spanish, Arabic, Portuguese, Dutch, Italian
 
if folder_name == "English":
    custom_alp = en
elif folder_name == "Turkish":
    custom_alp = tr
elif folder_name == "German":
    custom_alp = de
elif folder_name == "French":
    custom_alp = fr
elif folder_name == "Spanish":
    custom_alp = es
elif folder_name == "Arabic":
    custom_alp = ar
elif folder_name == "Portuguese":
    custom_alp = pt
elif folder_name == "Dutch":
    custom_alp = nl
elif folder_name == "Italian":
    custom_alp = it
else:
    pass
#custom_alp = tr  # en, tr, de, fr, es, ar, pt, nl, it
print(f"{folder_name}\n{custom_alp}") 

Turkish
re.compile('[abcçdefgğhıijklmnoöprsştuüvyzqxw]+', re.IGNORECASE)


#### Functions

In [15]:
def convert_one_character_letter(text):
    '''
    This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [17]:
def clean_text_pro(text, custom_alp): # for only string in specific language
    #text_clean = re.findall(custom_alp, text)
    text_clean = re.findall(custom_alp, str(text))
    text_result = " ".join(text_clean)
    return text_result

In [24]:
def rename_edit_file(file_path, rename_edit_folder="Rename", old_file_del = False, rename_file_orj = True, file_name_ref = "Sample", file_ref_start = 1):
    '''
    rename_edit_file(file_path, rename_edit_folder="Rename", old_file_del = False, rename_file_orj = True, file_name_ref = "Sample", file_ref_start = 1)\n
    '''
    file_path_var = os.path.dirname(f"{file_path}")
    file_path_rename = os.path.join(file_path_var, rename_edit_folder)
    Path(file_path_rename).mkdir(parents=True, exist_ok=True)
    file_list = os.listdir(file_path)

    for file in file_list:
    # file and extention
        file_without_ext = os.path.splitext(file)[0]
        file_ext = os.path.splitext(file)[1]
        # file clean and join
        file_without_ext_one = convert_one_character_letter(file_without_ext)
        file_without_ext_clean = clean_text_pro(file_without_ext_one, custom_alp)
        # filename tokenize
        word_list = word_tokenize(file_without_ext_clean)
        file_without_ext_new = " ".join(word_list)
        #file_new = f"{file_without_ext_new.lower()}{file_ext}"
        file_new = f"{file_without_ext_new.title()}{file_ext}"  # title() for each words begin capital letter
        file_new_var = f"{file_name_ref.title()}{file_ref_start}{file_ext}"
        # check move rename delete
        if isfile(f"{file_path_rename}/{file_new}"):
            pass
        else:
            source_file = f"{file_path}/{file}"
            destination_path = f"{file_path_rename}"        
            shutil.copy2(source_file, destination_path)
            if old_file_del:
                os.remove(source_file)
            else:
                pass
            if rename_file_orj:
                source_dest_file = f"{destination_path}/{file}"
                target_dest_file = f"{destination_path}/{file_new}"
                os.rename(source_dest_file, target_dest_file)
            else:
                source_dest_file = f"{destination_path}/{file}"
                target_dest_file = f"{destination_path}/{file_new_var}"
                os.rename(source_dest_file, target_dest_file)
        file_ref_start +=1

#### Files

In [ ]:
file_path = "C:\\Users\\Ertuğrul\\Documents\\Spec\\Yeni"  # r"C:\Users\Ertuğrul\Documents"
file_path

'C:\\Users\\Ertuğrul\\Documents\\Spec\\Yeni'

In [28]:
rename_edit_file(file_path, rename_edit_folder="Images", old_file_del = False, rename_file_orj = False, file_name_ref = "Sample", file_ref_start = 1)

#### Temp

In [ ]:
file_path = r"C:\Users\Ertuğrul\Documents"

In [ ]:
file_path_var = os.path.dirname(f"{file_path}")
file_path_var

'C:\\Users\\Ertuğrul'

In [ ]:
rename_edit_folder="Raname"

In [ ]:
file_path_rename = os.path.join(file_path_var, rename_edit_folder)
file_path_rename

'C:\\Users\\Ertuğrul\\Raname'